In [1]:
%pip install prophet

In [2]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm

# Configuración de la semilla para reproducibilidad
SEED = 47
np.random.seed(SEED)

# Configuraciones de Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Librerías importadas y semilla configurada.")

Librerías importadas y semilla configurada.


In [3]:
# Rutas de los archivos de entrada
SELLIN_PATH = 'https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/sell-in.txt.gz'
PRODUCTOS_PATH = 'https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_productos.txt'
STOCKS_PATH = 'https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_stocks.txt'
PRODUCTS_TO_PREDICT_PATH = 'https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt'

print("Cargando dataset de Sell-In...")
sellin_df = pd.read_csv(SELLIN_PATH, sep='\t', compression='gzip', dtype={'periodo': str, 'customer_id': str, 'product_id': str})
print(f"Sell-In cargado. Dimensiones: {sellin_df.shape}")

print("\nCargando dataset de Productos...")
productos_df = pd.read_csv(PRODUCTOS_PATH, sep='\t', dtype={'product_id': str})
print(f"Productos cargado. Dimensiones: {productos_df.shape}")

print("\nCargando dataset de Stocks...")
stocks_df = pd.read_csv(STOCKS_PATH, sep='\t', dtype={'periodo': str, 'product_id': str})
print(f"Stocks cargado. Dimensiones: {stocks_df.shape}")

print("\nCargando dataset de IDs de Productos a Predecir...")
productos_a_predecir_df = pd.read_csv(PRODUCTS_TO_PREDICT_PATH, sep='\t', dtype={'product_id': str})
print(f"IDs a predecir cargado. Dimensiones: {productos_a_predecir_df.shape}")

Cargando dataset de Sell-In...
Sell-In cargado. Dimensiones: (2945818, 7)

Cargando dataset de Productos...
Productos cargado. Dimensiones: (1251, 7)

Cargando dataset de Stocks...
Stocks cargado. Dimensiones: (13691, 3)

Cargando dataset de IDs de Productos a Predecir...
IDs a predecir cargado. Dimensiones: (780, 1)


In [4]:
print("\n--- Realizando Joins ---")
sellin_prod_df = pd.merge(sellin_df, productos_df, on='product_id', how='left')
all_chunk_df = pd.merge(sellin_prod_df, stocks_df, on=['periodo', 'product_id'], how='left')
print(f"Joins completados. Dimensiones de all_chunk_df: {all_chunk_df.shape}")

print("\nConvirtiendo columna 'periodo' a datetime...")
all_chunk_df['periodo'] = pd.to_datetime(all_chunk_df['periodo'], format='%Y%m', errors='coerce')
print("Conversión de 'periodo' completada.")
print(all_chunk_df.head(2))


--- Realizando Joins ---
Joins completados. Dimensiones de all_chunk_df: (2945818, 14)

Convirtiendo columna 'periodo' a datetime...
Conversión de 'periodo' completada.
     periodo customer_id product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn cat1     cat2        cat3      brand  sku_size    descripcion  stock_final
0 2017-01-01       10234      20524                      0                 2          0.05300  0.05300   HC  VAJILLA  Cristalino  Importado     500.0  Abrillantador          NaN
1 2017-01-01       10032      20524                      0                 1          0.13628  0.13628   HC  VAJILLA  Cristalino  Importado     500.0  Abrillantador          NaN


In [5]:
print("\n--- Filtrando DataFrame por productos a predecir ---")
all_chunk_depurado_df = all_chunk_df[all_chunk_df['product_id'].isin(productos_a_predecir_df['product_id'])].copy()
print(f"Filtrado completado. Dimensiones de all_chunk_depurado_df: {all_chunk_depurado_df.shape}")
print(f"product_id únicos en productos_a_predecir_df: {productos_a_predecir_df['product_id'].nunique()}")
print(f"product_id únicos en all_chunk_depurado_df: {all_chunk_depurado_df['product_id'].nunique()}")


--- Filtrando DataFrame por productos a predecir ---
Filtrado completado. Dimensiones de all_chunk_depurado_df: (2293481, 14)
product_id únicos en productos_a_predecir_df: 780
product_id únicos en all_chunk_depurado_df: 780


In [6]:
print("\n--- EDA Inicial sobre all_chunk_depurado_df ---")
print(f"1. Dimensiones: {all_chunk_depurado_df.shape}")
print("\n2. Información general:")
all_chunk_depurado_df.info(show_counts=True)
print(f"\n3. Conteo de valores nulos (solo columnas con nulos):")
null_counts = all_chunk_depurado_df.isnull().sum()
print(null_counts[null_counts > 0].sort_values(ascending=False))
if 'periodo' in all_chunk_depurado_df.columns and not all_chunk_depurado_df['periodo'].isnull().all():
    min_date = all_chunk_depurado_df['periodo'].min()
    max_date = all_chunk_depurado_df['periodo'].max()
    print(f"\n4. Rango de fechas: {min_date.strftime('%Y-%m-%d') if pd.notnull(min_date) else 'N/A'} a {max_date.strftime('%Y-%m-%d') if pd.notnull(max_date) else 'N/A'}")


--- EDA Inicial sobre all_chunk_depurado_df ---
1. Dimensiones: (2293481, 14)

2. Información general:
<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entries, 0 to 2945817
Data columns (total 14 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   periodo                2293481 non-null  datetime64[ns]
 1   customer_id            2293481 non-null  object        
 2   product_id             2293481 non-null  object        
 3   plan_precios_cuidados  2293481 non-null  int64         
 4   cust_request_qty       2293481 non-null  int64         
 5   cust_request_tn        2293481 non-null  float64       
 6   tn                     2293481 non-null  float64       
 7   cat1                   2293481 non-null  object        
 8   cat2                   2293481 non-null  object        
 9   cat3                   2293481 non-null  object        
 10  brand                  2293481 non-null  object    

In [7]:
print("\n--- Agregando datos de ventas por producto y mes ---")
df_ventas_mensuales_producto = all_chunk_depurado_df.groupby(['periodo', 'product_id'])['tn'].sum().reset_index()
df_ventas_mensuales_producto = df_ventas_mensuales_producto.rename(columns={'tn': 'total_tn_mes'})
print(f"Agregación completada. Dimensiones: {df_ventas_mensuales_producto.shape}")
print(df_ventas_mensuales_producto.head(2))


--- Agregando datos de ventas por producto y mes ---
Agregación completada. Dimensiones: (22349, 3)
     periodo product_id  total_tn_mes
0 2017-01-01      20001     934.77222
1 2017-01-01      20002     550.15707


In [8]:
print("\n--- Preparando función para procesar series de productos ---")

if not df_ventas_mensuales_producto.empty:
    GLOBAL_MIN_DATE = df_ventas_mensuales_producto['periodo'].min()
    GLOBAL_MAX_DATE = df_ventas_mensuales_producto['periodo'].max() # Esta es la última fecha CON DATOS
    print(f"Rango de fechas global en datos agregados: {GLOBAL_MIN_DATE.strftime('%Y-%m-%d')} a {GLOBAL_MAX_DATE.strftime('%Y-%m-%d')}")
else:
    GLOBAL_MIN_DATE = pd.to_datetime('2017-01-01')
    GLOBAL_MAX_DATE = pd.to_datetime('2019-12-01')
    print(f"Advertencia: df_ventas_mensuales_producto vacío. Usando fechas globales por defecto.")

def preparar_serie_producto(df_ventas_prod_especifico, producto_id, global_min_hist_date, global_max_hist_date):
    if df_ventas_prod_especifico.empty:
        date_idx_completo = pd.date_range(start=global_min_hist_date, end=global_max_hist_date, freq='MS')
        serie_completa = pd.Series(0, index=date_idx_completo) # No asignar nombre aquí aún
        serie_completa.index.name = 'periodo'
        return serie_completa

    serie_producto = df_ventas_prod_especifico.set_index('periodo')['total_tn_mes']
    product_actual_min_date = serie_producto.index.min()
    date_idx_producto_activo = pd.date_range(start=product_actual_min_date, end=global_max_hist_date, freq='MS')
    serie_completa = serie_producto.reindex(date_idx_producto_activo, fill_value=0)

    # Asignar nombres DESPUÉS de todas las operaciones de reindexado
    serie_completa.name = 'total_tn_mes'
    serie_completa.index.name = 'periodo'
    return serie_completa

print("Función 'preparar_serie_producto' definida.")


--- Preparando función para procesar series de productos ---
Rango de fechas global en datos agregados: 2017-01-01 a 2019-12-01
Función 'preparar_serie_producto' definida.


In [9]:
print("\n--- Procesando series para todos los productos a predecir ---")
series_procesadas_dict = {}
productos_sin_ventas_historicas = []

for product_id_objetivo in tqdm(productos_a_predecir_df['product_id'].unique(), desc="Procesando series"):
    df_ventas_este_producto = df_ventas_mensuales_producto[df_ventas_mensuales_producto['product_id'] == product_id_objetivo]
    serie_procesada = preparar_serie_producto(df_ventas_este_producto, product_id_objetivo, GLOBAL_MIN_DATE, GLOBAL_MAX_DATE)
    series_procesadas_dict[product_id_objetivo] = serie_procesada
    if df_ventas_este_producto.empty:
        productos_sin_ventas_historicas.append(product_id_objetivo)

print(f"Total de series procesadas: {len(series_procesadas_dict)}")
if productos_sin_ventas_historicas:
    print(f"Se encontraron {len(productos_sin_ventas_historicas)} productos sin ventas (serie de ceros creada).")


--- Procesando series para todos los productos a predecir ---


Procesando series:   0%|          | 0/780 [00:00<?, ?it/s]

Total de series procesadas: 780


In [10]:
from prophet import Prophet

print("\n--- Modelado (Kaggle Submission): Entrenar hasta Dic 2019, Predecir Feb 2020 ---")
start_time_modeling_kaggle = time.time()

# Para Kaggle, el último dato es GLOBAL_MAX_DATE (ej. 2019-12-01)
# Queremos predecir el segundo mes futuro (Febrero 2020)
N_PERIODOS_A_PREDECIR_KAGGLE = 2
TARGET_PREDICTION_INDEX_KAGGLE = 1 # 0 para el primer mes futuro (Ene), 1 para el segundo (Feb)

submission_list_kaggle = [] # Se llenará con {'product_id': product_id, 'tn': prediccion_target_month}
modelos_kaggle_fallidos = []

for product_id in tqdm(productos_a_predecir_df['product_id'].unique(), desc="Modelando para Kaggle"):
    serie_a_modelar = series_procesadas_dict.get(product_id)

    if serie_a_modelar is None: # No debería ocurrir
        submission_list_kaggle.append({'product_id': product_id, 'tn': 0.0})
        modelos_kaggle_fallidos.append(product_id)
        continue

    df_prophet_train_kaggle = serie_a_modelar.reset_index()
    df_prophet_train_kaggle = df_prophet_train_kaggle.rename(columns={'periodo': 'ds', 'total_tn_mes': 'y'})

    prediccion_final_producto = 0.0 # Default
    if len(df_prophet_train_kaggle) < 3 or df_prophet_train_kaggle['y'].nunique() == 1:
        if not df_prophet_train_kaggle.empty and df_prophet_train_kaggle['y'].nunique() == 1:
            prediccion_final_producto = df_prophet_train_kaggle['y'].iloc[0] # Predice la constante
            # Asegurar no negatividad para la predicción constante
            prediccion_final_producto = max(0, prediccion_final_producto)
        # Si es muy corta o vacía, la prediccion_final_producto ya es 0.0
        is_trivial_zeros = not df_prophet_train_kaggle.empty and df_prophet_train_kaggle['y'].nunique() == 1 and df_prophet_train_kaggle['y'].iloc[0] == 0
        if not is_trivial_zeros:
            modelos_kaggle_fallidos.append(product_id)
    else:
        try:
            m_kaggle = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False, seasonality_mode='additive')
            m_kaggle.fit(df_prophet_train_kaggle)
            future_df_kaggle = m_kaggle.make_future_dataframe(periods=N_PERIODOS_A_PREDECIR_KAGGLE, freq='MS')
            forecast_df_kaggle = m_kaggle.predict(future_df_kaggle)

            # Seleccionar la predicción del mes objetivo (el segundo mes)
            prediccion_array_2meses = forecast_df_kaggle['yhat'].iloc[-N_PERIODOS_A_PREDECIR_KAGGLE:].values
            prediccion_target_month = prediccion_array_2meses[TARGET_PREDICTION_INDEX_KAGGLE]
            prediccion_final_producto = max(0, prediccion_target_month) # Asegurar no negatividad

        except Exception as e:
            # print(f"  ERROR (Kaggle) procesando {product_id}: {e}. Usando 0.0.")
            modelos_kaggle_fallidos.append(product_id)
            # prediccion_final_producto ya es 0.0

    submission_list_kaggle.append({'product_id': product_id, 'tn': prediccion_final_producto})

print(f"\nModelado (Kaggle) completado en {time.time() - start_time_modeling_kaggle:.2f} segundos.")
if modelos_kaggle_fallidos:
    print(f"Número de productos (Kaggle) con fallback/fallo no trivial: {len(set(modelos_kaggle_fallidos))}")

df_submission_kaggle = pd.DataFrame(submission_list_kaggle)
df_submission_kaggle = df_submission_kaggle[['product_id', 'tn']] # Asegurar orden


--- Modelado (Kaggle Submission): Entrenar hasta Dic 2019, Predecir Feb 2020 ---


Modelando para Kaggle:   0%|          | 0/780 [00:00<?, ?it/s]

Se truncaron las últimas líneas 5000 del resultado de transmisión.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp25kt1ppq/g8dh8obx.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp25kt1ppq/_7ea2_wp.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=67795', 'data', 'file=/tmp/tmp25kt1ppq/g8dh8obx.json', 'init=/tmp/tmp25kt1ppq/_7ea2_wp.json', 'output', 'file=/tmp/tmp25kt1ppq/prophet_modelc1k9o0jh/prophet_model-20250720035805.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
03:58:05 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
03:58:05 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp25kt1ppq/5_sgl8v_.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp25kt1ppq/og5l5220.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running Cm


Modelado (Kaggle) completado en 851.22 segundos.


In [11]:
print("\n--- Preparación del Archivo de Submission para Kaggle ---")
print(f"El DataFrame de submission (Kaggle) contiene predicciones para {len(df_submission_kaggle)} productos.")
print(df_submission_kaggle.head())
submission_filename_kaggle = 'submission_prophet_febrero.csv' # Nombre más específico
df_submission_kaggle.to_csv(submission_filename_kaggle, index=False, header=True)
print(f"Archivo de submission '{submission_filename_kaggle}' guardado.")
# from google.colab import files
# files.download(submission_filename_kaggle)


--- Preparación del Archivo de Submission para Kaggle ---
El DataFrame de submission (Kaggle) contiene predicciones para 780 productos.
  product_id           tn
0      20001  1362.070852
1      20002  1117.738591
2      20003   624.334799
3      20004   230.298529
4      20005   279.618412
Archivo de submission 'submission_prophet_febrero.csv' guardado.


In [12]:
print("\n--- Fase de Modelado (Backtesting con Depuración DETALLADA): Entrenamiento y Predicción con Prophet ---")
start_time_modeling_backtest = time.time()

FECHA_FIN_ENTRENAMIENTO_BACKTEST = pd.to_datetime('2019-10-01')
N_PERIODOS_A_PREDECIR_BACKTEST = 2
TARGET_PREDICTION_INDEX_BACKTEST = 1

predicciones_backtest_target_month_dict = {}
modelos_backtest_fallidos = []

print(f"Backtesting: Entrenando con datos hasta {FECHA_FIN_ENTRENAMIENTO_BACKTEST.strftime('%Y-%m-%d')}")
print(f"Objetivo: Predecir el mes {TARGET_PREDICTION_INDEX_BACKTEST + 1} después del fin de entrenamiento.")

# --- MODIFICACIÓN: Ejecutar solo para unos pocos IDs para depurar ---
#lista_ids_depuracion_backtest = list(productos_a_predecir_df['product_id'].unique())[:5] # Probar con 5
#print(f"Depurando backtest con los siguientes IDs: {lista_ids_depuracion_backtest}")

#for product_id in tqdm(lista_ids_depuracion_backtest, desc="Modelando para Backtest"):
for product_id in tqdm(productos_a_predecir_df['product_id'].unique(), desc="Modelando para Backtest"):
    print(f"\n--- Procesando Product ID (Backtest): {product_id} ---")
    serie_completa_producto = series_procesadas_dict.get(product_id)
    prediccion_final_producto_backtest = 0.0

    if serie_completa_producto is None:
        print(f"  Advertencia: No se encontró serie procesada para product_id {product_id}. Usando 0.0.")
        predicciones_backtest_target_month_dict[product_id] = prediccion_final_producto_backtest
        modelos_backtest_fallidos.append(product_id)
        continue

    serie_entrenamiento = serie_completa_producto[serie_completa_producto.index <= FECHA_FIN_ENTRENAMIENTO_BACKTEST].copy()

    # ----- PRINTS DE DEPURACIÓN DETALLADOS -----
    print(f"  Para Product ID {product_id}:")
    if serie_entrenamiento.empty:
        print(f"    ERROR: serie_entrenamiento está VACÍA después de filtrar por fecha <= {FECHA_FIN_ENTRENAMIENTO_BACKTEST.strftime('%Y-%m-%d')}")
    else:
        print(f"    serie_entrenamiento: {len(serie_entrenamiento)} puntos. Rango: {serie_entrenamiento.index.min().strftime('%Y-%m-%d')} a {serie_entrenamiento.index.max().strftime('%Y-%m-%d')}")
        # Forzar nombres de índice y serie ANTES de reset_index
        serie_entrenamiento.index.name = 'periodo'
        serie_entrenamiento.name = 'total_tn_mes'
    # ----- FIN PRINTS DE DEPURACIÓN DETALLADOS (Parte 1) -----

    df_prophet_train_backtest = serie_entrenamiento.reset_index()
    df_prophet_train_backtest = df_prophet_train_backtest.rename(columns={'periodo': 'ds', 'total_tn_mes': 'y'})

    # ----- PRINTS DE DEPURACIÓN DETALLADOS (Parte 2) -----
    if not df_prophet_train_backtest.empty:
        print(f"    df_prophet_train_backtest (después de rename): {len(df_prophet_train_backtest)} filas. Columnas: {list(df_prophet_train_backtest.columns)}")
        print(f"    Valores únicos en 'y' de df_prophet_train_backtest: {df_prophet_train_backtest['y'].nunique() if 'y' in df_prophet_train_backtest else 'Columna y no existe'}")
        print(f"    Primeras 3 filas de df_prophet_train_backtest:\n{df_prophet_train_backtest.head(3)}")
    else:
        print(f"    ERROR: df_prophet_train_backtest está VACÍO.")
    # ----- FIN PRINTS DE DEPURACIÓN DETALLADOS (Parte 2) -----


    if len(df_prophet_train_backtest) < 3 or ('y' in df_prophet_train_backtest and df_prophet_train_backtest['y'].nunique() == 1) or 'y' not in df_prophet_train_backtest:
        print(f"    INFO FALLBACK para {product_id}: Serie de entrenamiento corta ({len(df_prophet_train_backtest)} ptos) o constante ({df_prophet_train_backtest['y'].nunique() if 'y' in df_prophet_train_backtest and not df_prophet_train_backtest.empty else 'N/A'} val. únicos) o 'y' no existe. Usando fallback.")
        if not df_prophet_train_backtest.empty and 'y' in df_prophet_train_backtest and df_prophet_train_backtest['y'].nunique() == 1:
            prediccion_final_producto_backtest = df_prophet_train_backtest['y'].iloc[0]
            prediccion_final_producto_backtest = max(0, prediccion_final_producto_backtest)
            print(f"      Fallback por constante: prediciendo {prediccion_final_producto_backtest}")
        else:
            # prediccion_final_producto_backtest ya es 0.0
            print(f"      Fallback por muy corta, vacía o sin columna 'y': prediciendo {prediccion_final_producto_backtest}")

        is_trivial_zeros = not df_prophet_train_backtest.empty and 'y' in df_prophet_train_backtest and df_prophet_train_backtest['y'].nunique() == 1 and df_prophet_train_backtest['y'].iloc[0] == 0
        if not is_trivial_zeros:
             modelos_backtest_fallidos.append(product_id)
        predicciones_backtest_target_month_dict[product_id] = prediccion_final_producto_backtest # Guardar fallback
        continue # Saltar al siguiente producto

    try:
        m_backtest = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False, seasonality_mode='additive')
        m_backtest.fit(df_prophet_train_backtest) # Aquí Prophet necesita las columnas 'ds' e 'y'
        future_df_backtest = m_backtest.make_future_dataframe(periods=N_PERIODOS_A_PREDECIR_BACKTEST, freq='MS')
        forecast_df_backtest = m_backtest.predict(future_df_backtest)
        prediccion_array_2meses = forecast_df_backtest['yhat'].iloc[-N_PERIODOS_A_PREDECIR_BACKTEST:].values
        prediccion_target_month = prediccion_array_2meses[TARGET_PREDICTION_INDEX_BACKTEST]
        prediccion_final_producto_backtest = max(0, prediccion_target_month)
        print(f"    Predicción Prophet (Backtest) para {product_id} (mes {TARGET_PREDICTION_INDEX_BACKTEST+1}): {prediccion_final_producto_backtest}")

    except Exception as e:
        print(f"    ERROR (Fit/Predict) procesando product_id {product_id} con Prophet: {e}. Usando 0.0")
        # prediccion_final_producto_backtest ya es 0.0
        modelos_backtest_fallidos.append(product_id)

    predicciones_backtest_target_month_dict[product_id] = prediccion_final_producto_backtest

# ... (resto de la Celda 12: prints de resumen)
print(f"\nModelado (Backtest) completado en {time.time() - start_time_modeling_backtest:.2f} segundos.")
print(f"Total de predicciones (Backtest) para el mes objetivo generadas: {len(predicciones_backtest_target_month_dict)}")
if modelos_backtest_fallidos:
    unique_fallidos_backtest = list(set(modelos_backtest_fallidos))
    print(f"Número de productos (Backtest) donde el modelo Prophet falló o se usó fallback (no trivial): {len(unique_fallidos_backtest)}")
    if unique_fallidos_backtest:
        print(f"Primeros 5 IDs con fallback/fallo (Backtest): {unique_fallidos_backtest[:5]}")


--- Fase de Modelado (Backtesting con Depuración DETALLADA): Entrenamiento y Predicción con Prophet ---
Backtesting: Entrenando con datos hasta 2019-10-01
Objetivo: Predecir el mes 2 después del fin de entrenamiento.


Modelando para Backtest:   0%|          | 0/780 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmp25kt1ppq/1w7zen10.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp25kt1ppq/ir8lm0eg.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=65079', 'data', 'file=/tmp/tmp25kt1ppq/1w7zen10.json', 'init=/tmp/tmp25kt1ppq/ir8lm0eg.json', 'output', 'file=/tmp/tmp25kt1ppq/prophet_model73xxe02p/prophet_model-20250720040917.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
04:09:17 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing



--- Procesando Product ID (Backtest): 20001 ---
  Para Product ID 20001:
    serie_entrenamiento: 34 puntos. Rango: 2017-01-01 a 2019-10-01
    df_prophet_train_backtest (después de rename): 34 filas. Columnas: ['ds', 'y']
    Valores únicos en 'y' de df_prophet_train_backtest: 34
    Primeras 3 filas de df_prophet_train_backtest:
          ds           y
0 2017-01-01   934.77222
1 2017-02-01   798.01620
2 2017-03-01  1303.35771


04:09:17 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp25kt1ppq/ca0hns0x.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp25kt1ppq/imbex6r_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=67185', 'data', 'file=/tmp/tmp25kt1ppq/ca0hns0x.json', 'init=/tmp/tmp25kt1ppq/imbex6r_.json', 'output', 'file=/tmp/tmp25kt1ppq/prophet_modelgeoe2xwa/prophet_model-20250720040917.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
04:09:17 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


...

    Predicción Prophet (Backtest) para 21266 (mes 2): 0

--- Procesando Product ID (Backtest): 21267 ---
  Para Product ID 21267:
    serie_entrenamiento: 8 puntos. Rango: 2019-03-01 a 2019-10-01
    df_prophet_train_backtest (después de rename): 8 filas. Columnas: ['ds', 'y']
    Valores únicos en 'y' de df_prophet_train_backtest: 8
    Primeras 3 filas de df_prophet_train_backtest:
          ds        y
0 2019-03-01  0.21578
1 2019-04-01  0.12291
2 2019-05-01  0.24451


04:24:58 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:n_changepoints greater than number of observations. Using 5.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp25kt1ppq/1_a9jpck.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp25kt1ppq/x8t7en90.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=62479', 'data', 'file=/tmp/tmp25kt1ppq/1_a9jpck.json', 'init=/tmp/tmp25kt1ppq/x8t7en90.json', 'output', 'file=/tmp/tmp25kt1ppq/prophet_model72x6sp36/prophet_model-20250720042458.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
04:24:58 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


    Predicción Prophet (Backtest) para 21267 (mes 2): 0

--- Procesando Product ID (Backtest): 21276 ---
  Para Product ID 21276:
    serie_entrenamiento: 8 puntos. Rango: 2019-03-01 a 2019-10-01
    df_prophet_train_backtest (después de rename): 8 filas. Columnas: ['ds', 'y']
    Valores únicos en 'y' de df_prophet_train_backtest: 8
    Primeras 3 filas de df_prophet_train_backtest:
          ds        y
0 2019-03-01  0.12249
1 2019-04-01  0.10173
2 2019-05-01  0.09283


04:24:58 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


    Predicción Prophet (Backtest) para 21276 (mes 2): 0.4711296830066134

Modelado (Backtest) completado en 941.94 segundos.
Total de predicciones (Backtest) para el mes objetivo generadas: 780
Número de productos (Backtest) donde el modelo Prophet falló o se usó fallback (no trivial): 9
Primeros 5 IDs con fallback/fallo (Backtest): ['21214', '20962', '21087', '20686', '20995']


In [13]:
print("\n--- Cálculo del TFE para Backtesting (Evaluando Diciembre 2019) ---")
start_time_evaluation_backtest = time.time()
lista_errores_absolutos_backtest = []
lista_ventas_reales_backtest = []

# FECHAS PARA OBTENER ACTUALS:
# Si FECHA_FIN_ENTRENAMIENTO_BACKTEST fue '2019-10-01',
# y TARGET_PREDICTION_INDEX_BACKTEST fue 1 (el segundo mes),
# el mes que estamos evaluando es 2019-10-01 + 2 meses = 2019-12-01.
MES_A_EVALUAR_BACKTEST = pd.to_datetime('2019-12-01')

print(f"Evaluando predicciones para el mes: {MES_A_EVALUAR_BACKTEST.strftime('%Y-%m')}")

for product_id, prediccion_mes_target in predicciones_backtest_target_month_dict.items():
    serie_historica_completa = series_procesadas_dict.get(product_id)
    if serie_historica_completa is None: continue

    # Obtener la venta real para el MES_A_EVALUAR_BACKTEST
    if MES_A_EVALUAR_BACKTEST in serie_historica_completa.index:
        venta_real_mes_target = serie_historica_completa.loc[MES_A_EVALUAR_BACKTEST]
    else:
        # print(f"Advertencia (Evaluación) para {product_id}: No se encontró dato real para {MES_A_EVALUAR_BACKTEST.strftime('%Y-%m')}. Omitiendo.")
        continue # O asignar 0 si se considera que si no hay registro es 0 venta

    error_absoluto = np.abs(venta_real_mes_target - prediccion_mes_target)
    lista_errores_absolutos_backtest.append(error_absoluto)
    lista_ventas_reales_backtest.append(venta_real_mes_target)

if not lista_ventas_reales_backtest or sum(lista_ventas_reales_backtest) < 1e-9: # Evitar división por casi cero
    tfe_global_backtest = 0.0 if (not lista_errores_absolutos_backtest or sum(lista_errores_absolutos_backtest) < 1e-9) else float('inf')
    resultado_tfe = "TFE Global Backtest = 0.0 (No hubo ventas reales significativas o errores)" if tfe_global_backtest == 0.0 else "ADVERTENCIA TFE: Suma de ventas reales es cero (o casi cero) pero hay errores."
else:
    tfe_global_backtest = sum(lista_errores_absolutos_backtest) / sum(lista_ventas_reales_backtest)
    resultado_tfe = f"TFE Global Backtest = {tfe_global_backtest:.4f}"

print(f"\n{resultado_tfe} para el mes {MES_A_EVALUAR_BACKTEST.strftime('%Y-%m')}")
print(f"Cálculo de TFE (Backtest) completado en {time.time() - start_time_evaluation_backtest:.2f} segundos.")


--- Cálculo del TFE para Backtesting (Evaluando Diciembre 2019) ---
Evaluando predicciones para el mes: 2019-12

TFE Global Backtest = 0.7285 para el mes 2019-12
Cálculo de TFE (Backtest) completado en 0.04 segundos.


In [14]:
predicciones_backtest_target_month_dict_df = pd.DataFrame(list(predicciones_backtest_target_month_dict.items()), columns=['product_id', 'tn'])
predicciones_backtest_target_month_dict_df
predicciones_backtest_target_month_dict_df.to_csv('predicciones_backtest_target_month_dict.csv', index=False)
#